#### In this notebook I outline the Target Engineering method i found useful for training more accurate models.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
import time
import datetime
# visualize
import matplotlib.pyplot as plt
from matplotlib import pyplot
import warnings
warnings.simplefilter('ignore')
import os

In [ ]:
train = pd.read_csv('../input/g-research-crypto-forecasting/train.csv')
train2 = pd.read_csv('../input/g-research-crypto-forecasting/supplemental_train.csv')
train = pd.concat([train,train2])
train = train.drop(columns=['VWAP','Count','Open','Low','High'])
train = train.sort_values(by=['Asset_ID','timestamp'])

In [ ]:
asset_details = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')
ids = list(asset_details.Asset_ID)
weights = np.array(list(asset_details.Weight))
asset_details['Normalized_Weight'] = asset_details['Weight'] / asset_details['Weight'].sum()


In [ ]:
mintime = train['timestamp'].min()
maxtime = train['timestamp'].max()
all_timestamps1 = np.array([i for i in range(mintime, maxtime,60)])
all_timestamps2 = sorted(train['timestamp'][train['timestamp']>=mintime].unique())
targets1 = pd.DataFrame(index=all_timestamps1)
targets2 = pd.DataFrame(index=all_timestamps2)
betas1 = pd.DataFrame(index=all_timestamps1)

In [ ]:
targets1 = pd.DataFrame(index=all_timestamps1)
targets2 = pd.DataFrame(index=all_timestamps2)
price_column = 'Close'
for i in train['Asset_ID'].unique():
    print(i)
    asset = train[train.Asset_ID == i].set_index(keys='timestamp')
    asset = asset[~asset.index.duplicated()]
    price2 = pd.Series(index=all_timestamps2, data=asset[price_column])
    price1 = pd.Series(index=all_timestamps1, data=asset[price_column])
    price1 = price1.astype(float).fillna(method='ffill')
    weight_i = asset_details[asset_details['Asset_ID']==i]['Normalized_Weight'].iloc[0]
    targets1[f'{i}'] = (
        price1.shift(periods=-16) /
        price1.shift(periods=-1)
    ) - 1
    targets1[f'w{i}'] = targets1[f'{i}'] * weight_i

    targets2[f'{i}'] = (
        price2.shift(periods=-16) /
        price2.shift(periods=-1)
    ) - 1
    targets2[f'w{i}'] = targets2[f'{i}'] * weight_i
    
targets1['m1'] = targets1[[f'w{i}' for i in train['Asset_ID'].unique()]].fillna(0).sum(axis=1)
targets2['m1'] = 0
m1 = targets1['m1']
m2 = targets2['m1']

targets1  = targets1[[f'{i}' for i in train['Asset_ID'].unique()]]
num1 = targets1.multiply(m1.values, axis=0).rolling(3750).mean().values
denom1 = m1.multiply(m1.values, axis=0).rolling(3750).mean().values
beta1 = np.nan_to_num(num1.T / denom1, nan=0., posinf=0., neginf=0.)

targets1 = targets1 - (beta1 * m1.values).T

targets2  = targets2[[f'{i}' for i in train['Asset_ID'].unique()]]
num2 = targets2.multiply(m2.values, axis=0).rolling(3750).mean().values
denom2 = m2.multiply(m2.values, axis=0).rolling(3750).mean().values
beta2 = np.nan_to_num(num2.T / denom2, nan=0., posinf=0., neginf=0.)

targets2 = targets2 - (beta2 * m2.values).T

for j,i in enumerate(train['Asset_ID'].unique()):
    betas1[f'{i}'] = beta1[j,:]

In [ ]:
full = pd.DataFrame()
for i in train['Asset_ID'].unique():
    
    asset = train[train.Asset_ID == int(i)].set_index(keys='timestamp')
    if len(asset) > 400000:
        print(f'asset size {asset.shape[0]}')
        recreated1 = pd.Series(index=asset.index, data=targets1[f'{i}'])
        recreated2 = pd.Series(index=asset.index, data=targets2[f'{i}'])
        recreatedbeta1 = pd.Series(index=asset.index, data=betas1[f'{i}'])
        recreatedm1 = pd.Series(index=asset.index, data=m1)
        #isna = pd.Series(index=asset.index, data=targets[f'isna'])
        asset['m'] =  recreatedm1
        asset['Beta'] = recreatedbeta1
        asset['TargetBeta'] = recreated1
        asset['TargetZero'] = recreated2

        full = pd.concat([full, asset])


### We produce two targets
- TargetBeta - we forward fill any missing close values, which allows up to create a beta for all assets at all times. We avoid random time periods where beta == 0 due to missing values.
- TargetZero - we set beta & timestamp mean (m) component == 0.

In [ ]:
full.head()

In [ ]:
print('Difference between true target and reconstructed TargetBeta for Asset 4 (DOGE)')
print('In later time periods there is less missing data and so the difference between the TargetBeta and the true Target is close to 0.')
abs(full['TargetBeta'][full['Asset_ID']==4]-full['Target'][full['Asset_ID']==4]).plot()

In [ ]:
print('Difference between true target and reconstructed TargetZero for Asset 4 (DOGE)')
print('In earlier time periods there is more missing data and so the difference between the TargetZero and the true Target is close to 0.')
abs(full['TargetZero'][full['Asset_ID']==4]-full['Target'][full['Asset_ID']==4]).plot()